<a href="https://colab.research.google.com/github/arapgechina24-lgtm/nctirs-platform-v2/blob/main/ai_training/nctirs_colab_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
!git clone https://github.com/arapgechina24-lgtm/nctirs-platform-v2.git
%cd nctirs-platform-v2/ai_training
!pip install -r requirements.txt

# NCTIRS Platform V2 - AI Training Pipeline
This notebook automatically clones your repository, installs dependencies, and runs the entire training and export pipeline.

In [ ]:
# 1. Download dataset
!python download_datasets.py

# 2. Generate synthetic data if needed
!python generate_synthetic.py

In [ ]:
# 3. Train the model (Using the combined pipeline)
!python train_combined.py

  ✗ UNSW-NB15 not found at /content/nctirs-platform-v2/ai_training/data/unsw_nb15/unsw_nb15_full.csv
  ✗ CICIDS2017 not found at /content/nctirs-platform-v2/ai_training/data/cicids2017/cicids2017_full.csv

[3/3] Loading Synthetic dataset...
  ✓ Synthetic: 100000 samples (80000 normal, 20000 attack)
  Attack types: {'Botnet': 2000, 'WebAttack': 2000, 'PortScan': 2000, 'Infiltration': 2000, 'DNSTunnel': 2000, 'Heartbleed': 2000, 'BruteForce': 2000, 'DataExfil': 2000, 'C2Beacon': 2000, 'DDoS': 2000}

COMBINED DATASET: 100000 total samples
  Normal: 80000 (80.0%)
  Attack: 20000 (20.0%)
  Features: 46

Train: 69991 | Val: 14991 | Test: 14991
Model: 71,472 parameters | Device: cpu

TRAINING — 100 epochs, batch 256, LR 0.001

Epoch    1/100 | Train: 0.6824 acc=0.8991 | Val: 0.6082 acc=0.9984 | LR: 1.00e-03 | 24s
Epoch    5/100 | Train: 0.5705 acc=0.9987 | Val: 0.5559 acc=0.9993 | LR: 1.00e-03 | 119s
Epoch   10/100 | Train: 0.5493 acc=0.9993 | Val: 0.5386 acc=0.9994 | LR: 1.00e-03 | 236s


In [ ]:
# 4. Evaluate the trained models
!python evaluate.py

In [ ]:
# 5. Export weights for use in the web app
!python export_weights.py

In [ ]:
# 6. Download the generated weights to your local machine
from google.colab import files
import os

print("Zipping checkpoints...")
!zip -r model_checkpoints.zip checkpoints/

if os.path.exists('model_checkpoints.zip'):
    print("Downloading checkpoints...")
    files.download('model_checkpoints.zip')
else:
    print("Could not find model_checkpoints.zip")